In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split, ShuffleSplit
from classifiers import TransparentLogisticRegression, TransparentLinearRegression
from matplotlib import pylab as pl
from scipy.sparse import diags
from IPython import display
from scale import decision_tree_scale
from IPython.display import display, HTML
from ipy_table import *

In [2]:
def transform(X, mns, sstd, axis=0):
    
    X = np.asanyarray(X)
    mns = np.asanyarray(mns)
    
    if axis and mns.ndim < X.ndim:

        return ((X - np.expand_dims(mns, axis=axis)) /
                    np.expand_dims(sstd, axis=axis))
    else:

        return (X - mns) / sstd

In [3]:
def sigmoid_afterscale(C):
    
    return 1/(1+np.exp(-C))

In [4]:
def sigmoid(x, mean, std, w, C=0):
    ex = np.exp(-((x-mean)/std*w + C))
    return 1/(1+ex)

In [5]:
def sigmoid_binary(x, mean, std, w, C=0):
    ex = np.exp(-(x*w + C))
    return 1/(1+ex)

In [6]:
#hepatitis
# WorKed
dataset = "hepatitis.csv"
class_index = 19
num_cols = 20
classes= ['DIE', 'LIVE']


#heart-c
dataset = "heart-c.csv"
class_index = 22
num_cols = 23
classes= ['<50', '>50_1']

#heart-statlog
# WorKed
dataset = "heart-statlog.csv"
class_index = 13
num_cols = 14
classes= ['absent', 'present']

#credit-g
dataset = "credit-g.csv"
class_index = 61
num_cols = 62
classes= ['good', 'bad']

#cmc
# WorKed
dataset = "cmc.csv"
class_index = 21
num_cols = 22
classes= ['1', '2']

# car
dataset = "car.csv"
class_index = 21
num_cols = 22
classes = ['acc', 'unacc']

#cmc
# WorKed
dataset = "cmc.csv"
class_index = 21
num_cols = 22
classes= ['1', '2']


read_cols = [i for i in range(num_cols) if i != class_index]
file_path = "D:\\IIT_Master\\2016 Spring\\CS597\\uci\\uci\\uci-tar\\nominal\\"+dataset

print dataset

cmc.csv


In [7]:
X = np.loadtxt(file_path, dtype=float, delimiter=",", skiprows=1, \
                   usecols=read_cols)
y = np.loadtxt(file_path, dtype=int, delimiter=",", skiprows=1, \
                   usecols=(class_index,), converters={class_index: lambda x: classes.index(x)})

num_inst, num_feat = np.shape(X)
print "The shape of this data set:",np.shape(X)

The shape of this data set: (1473L, 21L)


In [8]:
# Determine binary features
num_features = X.shape[1]
non_binary = []
binary = []
for i in range(num_features):
    if len(np.unique(X[:,i])) != 2:
        non_binary.append(i)
    else:
        binary.append(i)
        
print binary

# for binary features, replace zeros with -1, assuming the other values are 1; a more correct way would check if this was true.
if len(binary) > 0:
    X_b = X[:,binary]
    X_b[X_b == 0] = -1
    X[:,binary] = X_b
    
    
if len(non_binary) == 0:
    print "123"
    raise BaseException("There are only binary features in this data set")

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [9]:
clf_ori = TransparentLogisticRegression()
clf_ss = TransparentLogisticRegression()
clf_ig = TransparentLogisticRegression()

X_ori = X.copy()
X_ss = X.copy()
X_ig = X.copy()


if len(non_binary) > 0:
    X_ss[:,non_binary]=scale(X_ss[:,non_binary])

if len(non_binary) > 0:
    X_ig[:,non_binary]=clf_ig.fit_transform(X_ig[:,non_binary],y)
    
print binary
print non_binary

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[0, 9]


In [10]:
clf_ss.fit(X_ss, y)
clf_ig.fit(X_ig, y)

TransparentLogisticRegression(C=1.0, class_weight=None, dual=False,
               fit_intercept=True, intercept_scaling=1, max_iter=100,
               multi_class='ovr', n_jobs=1, penalty='l2',
               random_state=None, solver='liblinear', tol=0.0001,
               verbose=0, warm_start=False)

In [11]:
w1 = clf_ss.coef_[0]
w2 = clf_ig.coef_[0]

print w1
print w2

bias1 = clf_ss.intercept_
bias2 = clf_ig.intercept_

print bias1, bias2

print 1/(1+np.exp(-bias1))
print 1/(1+np.exp(-bias2))
print sum(y)/float(len(y))

[-0.68399032 -0.31431044 -0.15536154  0.07115112  0.44928914 -0.11401204
  0.04212012  0.10372407  0.01893613  0.80782395 -0.21930149  0.0676718
 -0.03114373 -0.12667031  0.00513598  0.20344634 -0.21055489 -0.01007069
  0.07289046  0.1985034  -0.26491924]
[-1.1750501  -0.12686316  0.03438046  0.2604083   0.63597677  0.07729725
  0.23067903  0.29082664  0.20509945  1.21628778 -0.21719768  0.06867218
  0.15472369  0.06068127  0.19227612  0.39622128 -0.02032157  0.17850484
  0.26135433  0.38436477 -0.2605171 ]
[-0.02538414] [-0.40195118]
[ 0.49365431]
[ 0.40084364]
0.572980312288


In [12]:
max_list = np.max(X, axis = 0 )
min_list = np.min(X, axis = 0)

print max_list
print min_list

[ 49.   1.   1.   1.   1.   1.   1.   1.   1.  16.   1.   1.   1.   1.   1.
   1.   1.   1.   1.   1.   1.]
[ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]


In [13]:
mean1_ = np.zeros(shape = (1, num_feat))
mean1 = mean1_.flatten()

mean2_ = np.zeros(shape = (1, num_feat))
mean2 = mean2_.flatten()


mean1[non_binary] = np.mean(X[:,non_binary], axis = 0)
mean2[non_binary] = clf_ig.mns


std1_ = np.ones(shape = (1, num_feat))
std1 = std1_.flatten()

std2_ = np.ones(shape = (1, num_feat))
std2 = std2_.flatten()



std1[non_binary] = np.std(X[:,non_binary], axis = 0)
std2[non_binary] = clf_ig.sstd

print mean1
print mean2

[ 32.53835709   0.           0.           0.           0.           0.           0.
   0.           0.           3.26137135   0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.        ]
[ 44.5   0.    0.    0.    0.    0.    0.    0.    0.    0.5   0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0. ]
[ 8.2244516   1.          1.          1.          1.          1.          1.
  1.          1.          2.35774813  1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.        ]
[ 14.51628414   1.           1.           1.           1.           1.           1.
   1.           1.           3.63099821   1.           1.           1.           1.
   1.           1.           1.           1.           1.           1.           1.        ]


In [14]:
print non_binary

print max_list

print min_list

[0, 9]
[ 49.   1.   1.   1.   1.   1.   1.   1.   1.  16.   1.   1.   1.   1.   1.
   1.   1.   1.   1.   1.   1.]
[ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]


In [15]:
currentX = np.copy(min_list)
counter = 100

for j in range(counter):
    
    for i in range(num_feat):

        if i in binary:
            num_ = 2           
           
        else:
            num_ = 10
        
        xs = np.linspace(min_list[i], max_list[i] , num = num_)
        
        currentX_ss = np.copy(currentX)
        currentX_ss[non_binary] = transform(currentX[non_binary], mean1[non_binary], std1[non_binary])
        
        currentX_ig = np.copy(currentX)
        currentX_ig[non_binary] = transform(currentX[non_binary], mean2[non_binary], std2[non_binary])
        
        C1 = np.dot(currentX_ss, w1) + bias1 - currentX_ss[i] * w1[i]
        C2 = np.dot(currentX_ig, w2) + bias2 - currentX_ig[i] * w2[i]
        
        value = abs(sigmoid(xs, mean1[i], std1[i], w1[i], C1) - sigmoid(xs, mean2[i], std2[i], w2[i], C2))
    
        currentX[i] = xs[np.argmax(value)]
        
        print np.max(value), currentX

0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [ 16.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.   0.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.]
0.354117129442 [

In [16]:
# currentX = currentX

print currentX

currentX_result_ss = np.copy(currentX)
currentX_result_ig = np.copy(currentX)

currentX_result_ss[non_binary] = transform(currentX[non_binary], mean1[non_binary], std1[non_binary])
currentX_result_ig[non_binary] = transform(currentX[non_binary], mean2[non_binary], std2[non_binary])

print currentX_result_ss
print currentX_result_ig


print clf_ss.predict_proba(currentX_result_ss)
print clf_ig.predict_proba(currentX_result_ig)
print clf_ss.predict_proba(currentX_result_ss) - clf_ig.predict_proba(currentX_result_ig)

[ 16.          -1.          -1.          -1.          -1.          -1.          -1.
  -1.          -1.           1.77777778   1.           1.          -1.          -1.
  -1.          -1.          -1.          -1.          -1.          -1.          -1.        ]
[-2.01087658 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -0.62924176  1.          1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.         -1.        ]
[-1.96331236 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.          0.35190813  1.          1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.         -1.        ]
[[ 0.32039964  0.67960036]]
[[ 0.6836414  0.3163586]]
[[-0.36324176  0.36324176]]


C:\Python27\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Python27\lib\site-packages\sklearn